In [1]:
import pandas as pd

# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

## Descargar datos de fuente oficial (Min Econ)

In [2]:
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)

url_fuente = 'https://www.economia.gob.ar/download/infoeco/apendice3a.xlsx'
response = requests.get(url_fuente, verify = False)
open('./datos/apendice3a.xlsx', "wb").write(response.content)

2204742

In [3]:
## Leer Excel
EPH_puntual = pd.read_excel('./datos/apendice3a.xlsx', sheet_name = 'EPH Puntual')
TD = pd.read_excel('./datos/apendice3a.xlsx', sheet_name = 'TD 03-')

In [7]:
TD.iloc[70:90]#.loc[TD['45.2_IT_0_T_13'].str.contains('2015')]


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Índice,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
70,2012-12-01 00:00:00,IV.12,0.069,0.077,0.058,0.039,0.026,0.053,0.073,0.041,...,0.069,0.062,0.014,0.061,0.042,0.032,0.041,0.049,0.037,0.064
71,2013-01-01 00:00:00,I.13,0.079,0.089,0.068,0.041,0.031,0.069,0.082,0.064,...,0.081,0.042,0.023,0.054,0.046,0.063,0.043,0.101,0.052,0.078
72,NaN,II.13,0.072,0.076,0.068,0.051,0.025,0.058,0.085,0.066,...,0.077,0.089,0.019,0.085,0.048,0.078,0.049,0.101,0.044,0.049
73,NaN,III.13,0.068,0.074,0.062,0.048,0.021,0.054,0.076,0.059,...,0.052,0.057,0.024,0.058,0.042,0.072,0.03,0.08,0.061,0.043
74,NaN,IV.13,0.064,0.065,0.063,0.049,0.022,0.055,0.078,0.055,...,0.063,0.072,0.011,0.057,0.05,0.078,0.038,0.047,0.039,0.043
75,,I.14,0.071,0.077,0.064,0.048,0.032,0.061,0.076,0.066,...,0.06,0.083,0.012,0.072,0.058,0.085,0.044,0.085,0.051,0.036
76,NaN,II.14,0.071,0.079,0.062,0.043,0.026,0.063,0.075,0.05,...,0.056,0.086,0.016,0.076,0.04,0.055,0.042,0.043,0.079,0.03
77,NaN,III.14,0.075,0.079,0.071,0.059,0.031,0.055,0.09,0.056,...,0.074,0.087,0.036,0.074,0.039,0.067,0.032,0.069,0.068,0.047
78,NaN,IV.14,0.069,0.072,0.066,0.047,0.027,0.063,0.081,0.06,...,0.066,0.067,0.012,0.051,0.062,0.078,0.039,0.043,0.064,0.036
79,NaN,I.15,0.071,0.079,0.062,0.043,0.026,0.063,0.075,0.05,...,0.056,0.086,0.016,0.076,0.04,0.055,0.042,0.043,0.079,0.03


## Elegir Hoja y indicar nombre de variables a extraer

In [8]:
datos = TD
serie = '45.2_'

# O bien, por ejemplo, para tomar la de EPH puntual:
# datos = EPH_puntual
# serie = '42.3_'

## Extraer dataframe de la hoja de Excel

In [32]:
datos['Unnamed: 20'].pad()#.count()

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
       ...  
111    0.071
112    0.071
113    0.071
114    0.071
115    0.071
Name: Unnamed: 20, Length: 116, dtype: object

In [26]:
datos = datos.iloc[:, :10] # Tirar las colum

mask = datos.apply(lambda x: x.astype(str).str.contains(serie))

df = datos[mask.cumsum() == True].dropna(axis = 1, how = 'all').dropna()
df, df.columns = df[1:] , df.iloc[0]
col_tiempo = df.columns[0]

df[col_tiempo] = df[col_tiempo].str.split('.').apply(lambda x: x[1]+x[0])
df[col_tiempo] = df[col_tiempo].str.replace(' ', '')
df[col_tiempo] = df[col_tiempo].str.replace('III', 'Q3').str.replace('II', 'Q2').str.replace('IV', 'Q4').str.replace('I', 'Q1')

df[col_tiempo] = pd.to_datetime(df[col_tiempo])

df = df.set_index(col_tiempo)
df.tail()

30,45.2_ECTDT_0_T_33,45.2_ECTDTG_0_T_37,45.2_ECTDTCU_0_T_38,45.2_ECTDTNE_0_T_42,45.2_ECTDTNO_0_T_42,45.2_ECTDTRP_0_T_49,45.2_ECTDTP_0_T_43,45.2_ECTDTC_0_T_38,45.2_ECTDTPG_0_T_46,45.2_ECTDTGM_0_T_46,...,45.2_ECTDTGSF_0_T_47,45.2_ECTDTRC_0_T_44,45.2_ECTDTSR_0_T_44,45.2_ECTDTSN_0_T_45,45.2_ECTDTCR_0_T_52,45.2_ECTDTN_0_T_41,45.2_ECTDTRG_0_T_46,45.2_ECTDTURG_0_T_52,45.2_ECTDTRT_0_T_47,45.2_ECTDTVCP_0_T_57
45.2_IT_0_T_13,,,,,,,,,,,,,,,,,,,,,
2021-01-01,0.102,0.11,0.066,0.061,0.099,0.103,0.084,0.079,0.119,0.082,...,0.09,0.068,0.095,0.086,0.044,0.109,0.09,0.123,0.061,0.039
2021-04-01,0.096009,0.101938,0.06825,0.060731,0.087344,0.104124,0.073916,0.090066,0.105247,0.085309,...,0.089013,0.072951,0.133491,0.081496,0.026347,0.081887,0.082397,0.099239,0.10727,0.039952
2021-07-01,0.082,0.089,0.07,0.045,0.062,0.088,0.055,0.07,0.095,0.084,...,0.086,0.059,0.099,0.065,0.033,0.07,0.057,0.048,0.085,0.015
2021-10-01,0.07,0.074,0.06,0.046,0.058,0.074,0.045,0.046,0.082,0.068,...,0.063,0.057,0.069,0.074,0.024,0.059,0.037,0.054,0.056,0.016
2022-01-01,0.07,0.073,0.054,0.051,0.066,0.074,0.048,0.049,0.08,0.065,...,0.071,0.036,0.071,0.068,0.021,0.074,0.061,0.046,0.038,0.01


In [25]:
datos[mask.cumsum() == True].dropna(axis = 1, how = 'all').iloc[75:].count(1)

75     40
76     40
77     40
78     40
79     40
80     39
81     40
82     40
83     40
84     40
85     40
86     40
87     40
88     40
89     40
90     40
91     40
92     40
93     40
94     40
95     40
96     40
97     40
98     40
99     40
100    40
101    40
102    40
103    40
104    40
105    40
106    40
107    40
108     0
109     0
110     1
111     1
112     1
113     0
114     1
115     1
dtype: int64

### Tomar raiz compartida por las variables extraidas

In [6]:
from difflib import SequenceMatcher

## Nombre de variables compartido por todas las series del dataset actual (lo llamamos raiz):
raiz_ = df.columns[0]
for col in df.columns:
    match = SequenceMatcher(None, raiz_, col).find_longest_match(0, len(raiz_), 0, len(col))
    next_ = raiz_[match.a:match.a + match.size]
    raiz_ = next_
    
print(raiz_)

45.2_ECTDT


## Guardar CSV

In [7]:
df.to_csv('./datos/'+raiz_+'.csv')